In [3]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


from tensorflow.keras import models, Sequential, layers, regularizers, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory 
import tensorflow.keras.applications.resnet50 as resnet50
import tensorflow.keras.applications.inception_v3 as inception_v3

from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input


In [4]:
path_train = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/train'
path_test = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/test'

# Functions

In [5]:
# Function for plotting the loss and accuracy 
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

# Dataset

In [40]:
# TRAIN DATASET
train_dataset = image_dataset_from_directory(directory=path_train,
                                      labels='inferred',
                                      label_mode="categorical",
                                      validation_split=None,
                                      subset=None,
                                      seed=123,
                                      image_size=(224, 224),
                                      batch_size=32)   

# Preprocess X in the train_dataset
# train_ds_prepro = train_dataset.map(preprocess)

###########
# VALIDATION DATASET
# val_dataset = image_dataset_from_directory(directory=path_train,
#                                       labels='inferred',
#                                       label_mode="categorical",
#                                       validation_split=0.2,
#                                       subset="validation",
#                                       seed=123,
#                                       image_size=(224, 224),
#                                       batch_size=32)
                                         
# Preprocess X in the val_dataset
# validation_ds_prepro = val_dataset.map(preprocess)

############
# TEST DATASET
test_dataset = image_dataset_from_directory(directory=path_test,
                                            labels='inferred',
                                            label_mode="categorical",
                                            validation_split=None,
                                            subset=None,
                                            seed=123,
                                            image_size=(224, 224),
                                            batch_size=32) 

# Preprocess X in the test_dataset
# test_ds_prepro = test_dataset.map(preprocess)

Found 12000 files belonging to 120 classes.
Found 8580 files belonging to 120 classes.


In [42]:
image_batch_train, labels_batch_train = next(iter(train_dataset))
image_batch_train.shape, labels_batch_train.shape


(TensorShape([32, 224, 224, 3]), TensorShape([32, 120]))

# Load pre-trained ResNet50 and Inception_V3 models 

In [10]:
# Load the ResNet50 model pre-trained on ImageNet
base_model_resnet50 = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers to prevent them from being updated during training
base_model_resnet50.trainable = False

94765736/94765736 [==============================] - 3s 0us/step


In [12]:
# Load the Inception_V3 model pre-trained on ImageNet
base_model_inceptionv3 = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers to prevent them from being updated during training
base_model_inceptionv3.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


# Combine the pre-trained layers 

In [13]:
# Get the output from Resnet50
input1 = layers.Input(shape=(224,224,3))
x1 = resnet_preprocess_input(input1)
x1 = base_model_resnet50(x1)
x1 = layers.Flatten()(x1)
output_resnet50 = x1
output_resnet50

<KerasTensor: shape=(None, 100352) dtype=float32 (created by layer 'flatten')>

In [14]:
# Get the output from Inception_V3
input2 = layers.Input(shape=(224,224,3))
x2 = inception_preprocess_input(input2)
x2 = base_model_inceptionv3(x2)
x2 = layers.Flatten()(x2)
output_inceptionv3 = x2
output_inceptionv3


<KerasTensor: shape=(None, 51200) dtype=float32 (created by layer 'flatten_1')>

In [15]:
# Combine the outputs
combined_output = layers.concatenate([output_resnet50, output_inceptionv3])
combined_output 

<KerasTensor: shape=(None, 151552) dtype=float32 (created by layer 'concatenate_2')>

# The combined model

In [46]:
# Initialize and compile
inputs = [input1, input2]   #the same train data 

x = layers.Dense(100, activation="relu")(combined_output)
x = layers.Dense(100, activation="relu")(x)
pred = layers.Dense(120, activation="softmax")(x)

combined_model = Model(inputs=inputs, outputs=[pred, pred])

opt = optimizers.Adam(learning_rate = 1e-4)
combined_model.compile(optimizer=opt, 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

combined_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 224, 224, 3)  0          ['input_3[0][0]']                
 ingOpLambda)                                                                               

In [47]:
# Train the combined model 
MODEL = "combined_model"

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=5, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor = 0.1,
                       patience=3,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}.h5".format(MODEL),
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)

history = combined_model.fit(x=[image_batch_train, image_batch_train],
                             y=labels_batch_train,
                             validation_split=0.2, 
                             epochs=10,
                             callbacks=[es, lr, mcp],
                             batch_size=32,
                             verbose=1)

Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 10.3866 - dense_11_loss: 5.1933 - dense_11_1_loss: 5.1933 - dense_11_accuracy: 0.0400 - dense_11_1_accuracy: 0.0400 - val_loss: 10.2977 - val_dense_11_loss: 5.1488 - val_dense_11_1_loss: 5.1488 - val_dense_11_accuracy: 0.0000e+00 - val_dense_11_1_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
1/1 [==============================] - 0s 210ms/step - loss: 5.3226 - dense_11_loss: 2.6613 - dense_11_1_loss: 2.6613 - dense_11_accuracy: 0.3600 - dense_11_1_accuracy: 0.3600 - val_loss: 10.7097 - val_dense_11_loss: 5.3549 - val_dense_11_1_loss: 5.3549 - val_dense_11_accuracy: 0.0000e+00 - val_dense_11_1_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
1/1 [==============================] - 0s 184ms/step - loss: 2.0417 - dense_11_loss: 1.0209 - dense_11_1_loss: 1.0209 - dense_11_accuracy: 0.8400 - dense_11_1_accuracy: 0.8400 - val_loss: 11.7864 - val_dense_11_loss: 5.8932 - val_dense_11_1_loss: 5.8932 - val_dense_11_accuracy

In [ ]:
plot_history(combined_model, title="Model 3")

In [ ]:
# Evaluate MODEL 2 on the test dataset
res = combined_model.evaluate(preprocessed_test_dataset)
test_accuracy = res[-1]
print(f"test_accuracy_model_1 = {round(test_accuracy,2)*100} %")
print(f'Chance level: {1./120*100:.1f}%')